<a href="https://colab.research.google.com/github/LoTzuChin/113-1-FinancialBigData/blob/main/finbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 加載 FinBERT 模型
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [3]:
def preprocess(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return inputs

In [4]:
import torch
import torch.nn.functional as F

def analyze_sentiment(text):
    inputs = preprocess(text)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)
    return probabilities

In [5]:
def get_sentiment_label(probabilities):
    labels = ["Negative", "Neutral", "Positive"]
    scores = probabilities[0].tolist()
    sentiment = labels[torch.argmax(probabilities)]
    return sentiment, scores

In [6]:
def analyze_article_sentiment(text):
    probabilities = analyze_sentiment(text)
    sentiment, scores = get_sentiment_label(probabilities)
    print(f"Sentiment: {sentiment}")
    print(f"Scores: Negative={scores[0]:.4f}, Neutral={scores[1]:.4f}, Positive={scores[2]:.4f}")

# 測試文章
article = "Two top Senate Democrats have called for an investigation into Elon Musks reported contacts with Russian President Vladimir Putin and his top aides. The lawmakers have urged the Pentagon and Justice Department to determine whether Musks alleged relations with a US adversary while holding major government contracts puts national security at risk. The Wall Street Journal reported last month that the worlds richest person has had multiple, high level conversations with Putin since 2022, which the Kremlin has denied. Musk wrote on his X platform on Friday that he's going to find out whos making these accusations and nuke them."
analyze_article_sentiment(article)

Sentiment: Positive
Scores: Negative=0.1531, Neutral=0.0001, Positive=0.8468
